In [6]:
%run imports.ipynb
%run countries.ipynb

In [13]:
# Scrape Wikipedia for countries and medical facilities
url_wikipedia = "https://en.wikipedia.org/wiki/Lists_of_hospitals_in_Africa"
response_wikipedia = requests.get(url_wikipedia)
soup_wikipedia = BeautifulSoup(response_wikipedia.content, 'html.parser')
table_wikipedia = soup_wikipedia.find('table', {'class': 'wikitable'})

data_wikipedia = []
if table_wikipedia:
    for row in table_wikipedia.find_all('tr')[1:]:
        cols = row.find_all('td')
        if cols:
            country_name = cols[0].text.strip()
            facilities = cols[3].text.strip().replace(',', '')
            for key, variations in countries.items():
                if country_name in variations:
                    data_wikipedia.append({"Country": key, "Medical Facilities": facilities})

# Scrape Worldometers for population data
url_worldometers = "https://www.worldometers.info/world-population/population-by-country/"
response_worldometers = requests.get(url_worldometers)
soup_worldometers = BeautifulSoup(response_worldometers.content, 'html.parser')
rows_worldometers = soup_worldometers.find_all('tr')

data_population = []
for row in rows_worldometers:
    cols = row.find_all('td')
    if cols:
        country_name = cols[1].text.strip()
        population = cols[2].text.strip().replace(',', '')
        for key, variations in countries.items():
            if country_name in variations:
                data_population.append({"Country": key, "Population": population})

# Convert to DataFrame for merging (easy)
df_wikipedia = pd.DataFrame(data_wikipedia)
df_population = pd.DataFrame(data_population)

# Merge data on country name
merged_data = pd.merge(df_wikipedia, df_population, on="Country")

# Write to CSV
csvMFP2023 = 'medicalFacilitiesPopulation2023.csv'
merged_data.to_csv(csvMFP2023, index=False, encoding='utf-8')

#print(f'Data successfully saved to {csvMFP2023}.')

dfMFP2023 = pd.read_csv(csvMFP2023)

In [14]:
#Adding Density Med Fac 2023
dfMFP2023['Density Med Fac (100,000 pop)'] = dfMFP2023['Medical Facilities']/(dfMFP2023['Population']/100000)